In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [2]:
os.system('mkdir tacotron2-yasmin-alignment')

256

In [3]:
import tensorflow as tf
import numpy as np
from glob import glob
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import tacotron2_nvidia as tacotron2
import malaya_speech.config
import numpy as np
import json
import malaya_speech.train as train
import re



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
def norm_mean_std(x, mean, std):
    zero_idxs = np.where(x == 0.0)[0]
    x = (x - mean) / std
    x[zero_idxs] = 0.0
    return x

def average_by_duration(x, durs):
    mel_len = durs.sum()
    durs_cum = np.cumsum(np.pad(durs, (1, 0)))
    
    x_char = np.zeros((durs.shape[0],), dtype=np.float32)
    for idx, start, end in zip(range(mel_len), durs_cum[:-1], durs_cum[1:]):
        values = x[start:end][np.where(x[start:end] != 0.0)[0]]
        x_char[idx] = np.mean(values) if len(values) > 0 else 0.0

    return x_char.astype(np.float32)

In [5]:
f0_stat = np.load('../speech-bahasa/yasmin-stats/stats_f0.npy')
energy_stat = np.load('../speech-bahasa/yasmin-stats/stats_energy.npy')
mel_stat = np.load('../speech-bahasa/yasmin-stats/stats.npy')

In [6]:
with open('mels-yasmin.json') as fopen:
    files = json.load(fopen)
    
reduction_factor = 1
maxlen = 1280
minlen = 32
pad_to = 8
data_min = 1e-2

_pad = 'pad'
_start = 'start'
_eos = 'eos'
_punctuation = "!'(),.:;? "
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

MALAYA_SPEECH_SYMBOLS = (
    [_pad, _start, _eos] + list(_special) + list(_punctuation) + list(_letters)
)

def generate(files):
    for f in files:
        f = f.decode()
        mel = np.load(f)
        mel_length = len(mel)
        
        if mel_length > maxlen or mel_length < minlen:
            continue
            
        mel = norm_mean_std(mel, mel_stat[0], mel_stat[1])

        stop_token_target = np.zeros([len(mel)], dtype = np.float32)

        text_ids = np.load(f.replace('mels', 'text_ids'), allow_pickle=True)[
            0
        ]
        text_ids = ''.join([c for c in text_ids if c in MALAYA_SPEECH_SYMBOLS])
        text_ids = re.sub(r'[ ]+', ' ', text_ids).strip()
        text_input = np.array(
            [
                MALAYA_SPEECH_SYMBOLS.index(c)
                for c in text_ids
            ]
        )
        num_pad = pad_to - ((len(text_input) + 2) % pad_to)
        text_input = np.pad(
            text_input, ((1, 1)), 'constant', constant_values = ((1, 2))
        )
        text_input = np.pad(
            text_input, ((0, num_pad)), 'constant', constant_values = 0
        )
        num_pad = pad_to - ((len(mel) + 1) % pad_to) + 1
        pad_value_mel = np.log(data_min)
        mel = np.pad(
            mel,
            ((0, num_pad), (0, 0)),
            'constant',
            constant_values = pad_value_mel,
        )
        stop_token_target = np.pad(
            stop_token_target, ((0, num_pad)), 'constant', constant_values = 1
        )
        len_mel = [len(mel)]
        len_text_ids = [len(text_input)]
        
        
        f0 = np.load(f.replace('mels', 'f0s'))
        num_pad = pad_to - ((len(f0) + 1) % pad_to) + 1
        f0 = np.pad(
            f0,
            ((0, num_pad)),
            'constant',
        )
        f0 = norm_mean_std(f0, f0_stat[0], f0_stat[1])
        len_f0 = [len(f0)]
        
        energy = np.load(f.replace('mels', 'energies'))
        num_pad = pad_to - ((len(energy) + 1) % pad_to) + 1
        energy = np.pad(
            energy,
            ((0, num_pad)),
            'constant',
        )
        energy = norm_mean_std(energy, energy_stat[0], energy_stat[1])
        len_energy = [len(energy)]
        
        
        yield {
            'mel': mel,
            'text_ids': text_input,
            'len_mel': len_mel,
            'len_text_ids': len_text_ids,
            'stop_token_target': stop_token_target,
            'f0': f0,
            'len_f0': len_f0,
            'energy': energy,
            'len_energy': len_energy,
            'f': [f]
        }

def parse(example):
    mel_len = example['len_mel'][0]
    input_len = example['len_text_ids'][0]
    g = tacotron2.generate_guided_attention(mel_len, input_len, reduction_factor = reduction_factor)
    example['g'] = g
    return example
    
    
def get_dataset(files, batch_size = 32, shuffle_size = 32, thread_count = 24):
    def get():
        dataset = tf.data.Dataset.from_generator(
            generate,
            {
                'mel': tf.float32,
                'text_ids': tf.int32,
                'len_mel': tf.int32,
                'len_text_ids': tf.int32,
                'stop_token_target': tf.float32,
                'f0': tf.float32,
                'len_f0': tf.int32,
                'energy': tf.float32,
                'len_energy': tf.int32,
                'f': tf.string
            },
            output_shapes = {
                'mel': tf.TensorShape([None, 80]),
                'text_ids': tf.TensorShape([None]),
                'len_mel': tf.TensorShape([1]),
                'len_text_ids': tf.TensorShape([1]),
                'stop_token_target': tf.TensorShape([None]),
                'f0': tf.TensorShape([None]),
                'len_f0': tf.TensorShape([1]),
                'energy': tf.TensorShape([None]),
                'len_energy': tf.TensorShape([1]),
                'f': tf.TensorShape([1]),
            },
            args = (files,),
        )
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.padded_batch(
            shuffle_size,
            padded_shapes = {
                'mel': tf.TensorShape([None, 80]),
                'text_ids': tf.TensorShape([None]),
                'len_mel': tf.TensorShape([1]),
                'len_text_ids': tf.TensorShape([1]),
                'g': tf.TensorShape([None, None]),
                'stop_token_target': tf.TensorShape([None]),
                'f0': tf.TensorShape([None]),
                'len_f0': tf.TensorShape([1]),
                'energy': tf.TensorShape([None]),
                'len_energy': tf.TensorShape([1]),
                'f': tf.TensorShape([1]),
            },
            padding_values = {
                'mel': tf.constant(0, dtype = tf.float32),
                'text_ids': tf.constant(0, dtype = tf.int32),
                'len_mel': tf.constant(0, dtype = tf.int32),
                'len_text_ids': tf.constant(0, dtype = tf.int32),
                'g': tf.constant(-1.0, dtype = tf.float32),
                'stop_token_target': tf.constant(0, dtype = tf.float32),
                'f0': tf.constant(0, dtype = tf.float32),
                'len_f0': tf.constant(0, dtype = tf.int32),
                'energy': tf.constant(0, dtype = tf.float32),
                'len_energy': tf.constant(0, dtype = tf.int32),
                'f': tf.constant('', dtype = tf.string),
            },
        )
        return dataset

    return get

In [7]:
features = get_dataset(files['test'])()
features = features.make_one_shot_iterator().get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [8]:
input_ids = features['text_ids']
input_lengths = features['len_text_ids'][:, 0]
speaker_ids = tf.constant([0], dtype = tf.int32)
mel_outputs = features['mel']
mel_lengths = features['len_mel'][:, 0]
guided = features['g']
stop_token_target = features['stop_token_target']
batch_size = tf.shape(guided)[0]

In [9]:
model = tacotron2.Model(
    [input_ids, input_lengths],
    [mel_outputs, mel_lengths],
    len(MALAYA_SPEECH_SYMBOLS),
)



Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for u

In [10]:
r = model.decoder_logits['outputs']
decoder_output, post_mel_outputs, alignment_histories, _, _, _ = r
stop_token_predictions = model.decoder_logits['stop_token_prediction']

In [11]:
path = 'tacotron2-yasmin'
ckpt_path = tf.train.latest_checkpoint(path)
ckpt_path

'tacotron2-yasmin/model.ckpt-60000'

In [12]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [13]:
saver = tf.train.Saver()
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from tacotron2-yasmin/model.ckpt-60000


In [14]:
import matplotlib.pyplot as plt

In [15]:
def decode(x):
    return ''.join([MALAYA_SPEECH_SYMBOLS[i] for i in x])

In [16]:
def get_duration_from_alignment(alignment):
    D = np.array([0 for _ in range(np.shape(alignment)[0])])

    for i in range(np.shape(alignment)[1]):
        max_index = list(alignment[:, i]).index(alignment[:, i].max())
        D[max_index] = D[max_index] + 1

    return D

In [17]:
count = 0
while True:
    try:
        o = sess.run([decoder_output, post_mel_outputs, stop_token_predictions, alignment_histories, features])
        f = o[-1]
        for i in range(len(f['f'])):
            file = f['f'][i,0].decode().replace('../speech-bahasa/', '').replace('/', '-')
            file = f'tacotron2-yasmin-alignment/{file}'
            len_mel = f['len_mel'][i, 0]
            len_text_ids = f['len_text_ids'][i, 0]
            d = get_duration_from_alignment(o[3][i, :len_text_ids, :len_mel])
            assert d.sum() == len_mel
            np.save(file, d)
        count += 1
    except:
        break

In [22]:
!du -hs tacotron2-yasmin-alignment

419M	tacotron2-yasmin-alignment


In [26]:
len(glob('tacotron2-yasmin-alignment/*.npy'))

105890